<a href="https://colab.research.google.com/github/DiegoAnas/SNN-NLP/blob/master/Sentiment%20CNN%20Deeper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [10]:
import tensorflow
print(tensorflow.__version__)
import tensorflow.keras as keras

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import imdb
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, \
  Dense, Activation, GlobalMaxPooling1D, Add, Flatten, concatenate, Dropout

import numpy as np
from sklearn.model_selection import StratifiedKFold

2.1.0-rc1


In [0]:
# Parameters:
# Word Embedding
max_features = 10000
maxlen = 400
embedding_dims = 250

# Convolution
kernel_sizes = [3, 4, 5]  # or filter length
filters = 250 
pool_size = 4 

# Training
batch_size = 30
epochs = 5

In [4]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
x_data = np.concatenate((x_train, x_test))
y_data = np.concatenate((y_train, y_test))
skf = StratifiedKFold(n_splits=5, random_state=42)

Loading data...
17465344/17464789 [==============================] - 0s 0us/step
25000 train sequences
25000 test sequences
Pad sequences (samples x time)


In [0]:
# Network Architecture
def create_conv_mb_model(activation:str, batch_normalize:bool=False, dropout:bool=False,
                         dropout_rate:float=0.1, embedding_dropout:bool=False,
                         embedding_dropout_rate:float=0.1):
  # Extra layer Multi-branch
  var_input = Input(shape=(400,))
  emb_layer = Embedding(max_features,
                      embedding_dims,
                      input_length=maxlen)(var_input)
  if embedding_dropout:
    emb_layer = SpatialDropout1D(rate=embedding_dropout_rate)(emb_layer)
  conv_branches = []
  for kernel_size in kernel_sizes:
    conv_layer_1 = Conv1D(filters, kernel_size=kernel_size, padding='valid', 
                          activation=activation, strides=1)(emb_layer)
    if batch_normalize:
          conv_layer_1 = BatchNormalization()(conv_layer_1)
    conv_layer_2 = Conv1D(filters, kernel_size=kernel_size, padding='valid',
                          activation=activation, strides=1)(conv_layer_1)
    if batch_normalize:
          conv_layer_2 = BatchNormalization()(conv_layer_2)
    maxpooling = GlobalMaxPooling1D()(conv_layer_2)
    conv_branches.append(maxpooling)

  merge_layer = concatenate(conv_branches)
  if dropout:
      merge_layer = Dropout(rate=dropout_rate, seed=42)(merge_layer)
  dense_layer = Dense(1, activation='sigmoid')(merge_layer)
  model = Model(inputs=var_input, outputs=dense_layer)
  model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
  return model

In [11]:
for train, test in skf.split(x_data, y_data):
  target = create_conv_mb_model(activation='relu')
  target.fit(x_data[train], y_data[train],
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_data[test], y_data[test]))

Train on 40000 samples, validate on 10000 samples
Epoch 1/5
40000/40000 [==============================] - 57s 1ms/sample - loss: 0.3632 - accuracy: 0.8352 - val_loss: 0.3216 - val_accuracy: 0.8612
Epoch 2/5
40000/40000 [==============================] - 50s 1ms/sample - loss: 0.2499 - accuracy: 0.8960 - val_loss: 0.3481 - val_accuracy: 0.8522
Epoch 3/5
40000/40000 [==============================] - 50s 1ms/sample - loss: 0.1930 - accuracy: 0.9219 - val_loss: 0.2833 - val_accuracy: 0.8863
Epoch 4/5
40000/40000 [==============================] - 50s 1ms/sample - loss: 0.1375 - accuracy: 0.9464 - val_loss: 0.3052 - val_accuracy: 0.8842
Epoch 5/5
40000/40000 [==============================] - 50s 1ms/sample - loss: 0.0855 - accuracy: 0.9681 - val_loss: 0.4039 - val_accuracy: 0.8747
Train on 40000 samples, validate on 10000 samples
Epoch 1/5
40000/40000 [==============================] - 51s 1ms/sample - loss: 0.3582 - accuracy: 0.8349 - val_loss: 0.2913 - val_accuracy: 0.8754
Epoch 2/5
40

In [12]:
for train, test in skf.split(x_data, y_data):
  target = create_conv_mb_model(activation='selu')
  target.fit(x_data[train], y_data[train],
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_data[test], y_data[test]))

Train on 40000 samples, validate on 10000 samples
Epoch 1/5
40000/40000 [==============================] - 51s 1ms/sample - loss: 0.3520 - accuracy: 0.8420 - val_loss: 0.3099 - val_accuracy: 0.8675
Epoch 2/5
40000/40000 [==============================] - 51s 1ms/sample - loss: 0.2662 - accuracy: 0.8911 - val_loss: 0.2830 - val_accuracy: 0.8840
Epoch 3/5
40000/40000 [==============================] - 51s 1ms/sample - loss: 0.2110 - accuracy: 0.9165 - val_loss: 0.2909 - val_accuracy: 0.8875
Epoch 4/5
40000/40000 [==============================] - 51s 1ms/sample - loss: 0.1586 - accuracy: 0.9395 - val_loss: 0.3746 - val_accuracy: 0.8753
Epoch 5/5
40000/40000 [==============================] - 51s 1ms/sample - loss: 0.1264 - accuracy: 0.9519 - val_loss: 0.4094 - val_accuracy: 0.8837
Train on 40000 samples, validate on 10000 samples
Epoch 1/5
40000/40000 [==============================] - 52s 1ms/sample - loss: 0.3476 - accuracy: 0.8453 - val_loss: 0.2910 - val_accuracy: 0.8785
Epoch 2/5
40

In [0]:
# Network Architecture
def create_conv_mb_model_2(activation:str, batch_normalize:bool=False, dropout:bool=False, 
                           dropout_rate:float=0.1, embedding_dropout:bool=False, 
                           embedding_dropout_rate:float=0.1):
  # Extra layer Multi-branch w/o globalmaxpooling
  var_input = Input(shape=(400,))
  emb_layer = Embedding(max_features,
                      embedding_dims,
                      input_length=maxlen)(var_input)
  if embedding_dropout:
    emb_layer = SpatialDropout1D(rate=embedding_dropout_rate)(emb_layer)
  conv_branches = []
  for kernel_size in kernel_sizes:
    conv_layer_1 = Conv1D(filters, kernel_size=kernel_size, padding='valid', 
                          activation=activation, strides=1)(emb_layer)
    if batch_normalize:
          conv_layer_1 = BatchNormalization()(conv_layer_1)
    conv_layer_2 = Conv1D(filters, kernel_size=kernel_size, padding='valid',
                          activation=activation, strides=1)(conv_layer_1)
    if batch_normalize:
          conv_layer_2 = BatchNormalization()(conv_layer_2)
    maxpooling = MaxPooling1D(pool_size=3)(conv_layer_2)
    flattened = Flatten()(maxpooling)
    conv_branches.append(flattened)

  merge_layer = concatenate(conv_branches)
  if dropout:
      merge_layer = Dropout(rate=dropout_rate, seed=42)(merge_layer)
  dense_layer = Dense(1, activation='sigmoid')(merge_layer)
  model = Model(inputs=var_input, outputs=dense_layer)
  model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
  return model

In [0]:
epochs=2

In [17]:
for train, test in skf.split(x_data, y_data):
  target = create_conv_mb_model_2(activation='relu')
  target.fit(x_data[train], y_data[train],
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_data[test], y_data[test]))

Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 51s 1ms/sample - loss: 0.3556 - accuracy: 0.8372 - val_loss: 0.2886 - val_accuracy: 0.8782
Epoch 2/2
40000/40000 [==============================] - 51s 1ms/sample - loss: 0.2717 - accuracy: 0.8871 - val_loss: 0.2831 - val_accuracy: 0.8836
Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 52s 1ms/sample - loss: 0.3613 - accuracy: 0.8285 - val_loss: 0.3071 - val_accuracy: 0.8689
Epoch 2/2
40000/40000 [==============================] - 51s 1ms/sample - loss: 0.2711 - accuracy: 0.8885 - val_loss: 0.2971 - val_accuracy: 0.8792
Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 51s 1ms/sample - loss: 0.3690 - accuracy: 0.8230 - val_loss: 0.2959 - val_accuracy: 0.8765
Epoch 2/2
40000/40000 [==============================] - 51s 1ms/sample - loss: 0.2752 - accuracy: 0.8855 - va

In [18]:
for train, test in skf.split(x_data, y_data):
  target = create_conv_mb_model_2(activation='selu')
  target.fit(x_data[train], y_data[train],
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_data[test], y_data[test]))

Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 51s 1ms/sample - loss: 0.5485 - accuracy: 0.7797 - val_loss: 0.7152 - val_accuracy: 0.8064
Epoch 2/2
40000/40000 [==============================] - 51s 1ms/sample - loss: 1.0188 - accuracy: 0.8130 - val_loss: 1.3654 - val_accuracy: 0.8054
Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 52s 1ms/sample - loss: 0.5702 - accuracy: 0.7793 - val_loss: 0.7965 - val_accuracy: 0.7737
Epoch 2/2
40000/40000 [==============================] - 51s 1ms/sample - loss: 0.9231 - accuracy: 0.8176 - val_loss: 1.4665 - val_accuracy: 0.7924
Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 52s 1ms/sample - loss: 0.5624 - accuracy: 0.7825 - val_loss: 0.8221 - val_accuracy: 0.7972
Epoch 2/2
40000/40000 [==============================] - 51s 1ms/sample - loss: 0.9780 - accuracy: 0.8138 - va